In [1]:
!jupyter kernelspec list

Available kernels:
  pyspark        C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySpark
  pysparkauto    C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySparkAuto
  python3        C:\Anaconda3\envs\aiml\share\jupyter\kernels\python3


In [3]:
%pycat C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySpark\kernel.json
    

In [4]:
from pyspark.sql import *
import pyspark.sql.functions as F 


In [5]:
spark = SparkSession.builder.appName("basic").getOrCreate()

In [6]:
spark

In [ ]:
len(dir(DataFrame)), len(dir(Column)), len(dir(F))
file = 'data/spark/Cartier+for+WinnersCurse.csv'
sdf = spark.read.format('csv').option("header", "true")\
                .option("inferSchema", "true").load(file)